<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/time_vald.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==2.1.0-rc0

     |████████████████████████████████| 402.3MB 38kB/s 
     |████████████████████████████████| 3.8MB 49.9MB/s 
     |████████████████████████████████| 450kB 49.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=cf1706c758352d5d4ebb6f8587c314fb24fa48582c25ccbd18791d47e3cca672
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc3
    Uninstalling tensorflow-2.2.0rc3:
      Successfully uninstalled tensorflow-2.2.0rc

In [2]:
pip install keras==2.3.1

In [3]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 79.0MB/s]
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 106MB/s]
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 107MB/s]
 63% 33.0M/52.2M [00:01<00:00, 24.1MB/s]
100% 52.2M/52.2M [00:01<00:00, 46.9MB/s]
 70% 41.0M/58.3M [00:01<00:00, 27.3MB/s]
100% 58.3M/58.3M [00:01<00:00, 54.8MB/s]


In [4]:

from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [5]:

import pandas as pd
import numpy as np
#read autoenoded features
path = F"/content/gdrive/My Drive/autoenc.csv" 
ext=pd.read_csv(path,index_col=0)
ls=list(ext)
trn=pd.read_csv('train_transaction.csv.zip',usecols=['isFraud'])
ext=ext.loc[:trn.shape[0]]
ext['isFraud']=trn['isFraud']
import lightgbm as lgb
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
fnl=pd.DataFrame()
cols=[]
val=[]
trn.shape[0]

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


590540

In [0]:

ext=ext.dropna()
ext=ext.replace([np.inf,-np.inf],0)

In [7]:
trn=[]

#fit model with n rows and predict next 40000 rows for each column

for col in tqdm(ls):
    for i in range(50000,590540,40000):
        fit=ext.loc[0:i]
        pdct=ext.loc[i:i+40000]
        mod=lgb.LGBMClassifier(max_bins= 37,max_depth= 40,min_data_in_leaf= 110,min_gain_to_split= 1,num_leaves=48)
        mod.fit(fit[col].values.reshape(-1,1),fit['isFraud'])
        val.append(roc_auc_score(pdct['isFraud'],mod.predict_proba(pdct[col].values.reshape(-1,1))[:,1]))
        trn.append(roc_auc_score(fit['isFraud'],mod.predict_proba(fit[col].values.reshape(-1,1))[:,1]))
        cols.append(col)

100%|██████████| 256/256 [50:38<00:00, 11.87s/it]


In [0]:

fnl['cols']=cols
fnl['validation']=val
fnl['train']=trn
fnl['validation']=fnl.groupby(['cols'])['validation'].transform('mean')
fnl['train']=fnl.groupby(['cols'])['train'].transform('mean')
fnl=fnl[['validation','train','cols']]
fnl=fnl.drop_duplicates(keep='last')

In [9]:

fnl=fnl.sort_values(by='validation')
fnl.tail(100)['cols'].values

array(['Hidden_3_46', 'Hidden_3_202', 'Hidden_3_19', 'Hidden_3_146',
       'Hidden_3_134', 'Hidden_3_174', 'Hidden_3_84', 'Hidden_3_124',
       'Hidden_3_203', 'Hidden_3_121', 'Hidden_3_114', 'Hidden_3_118',
       'Hidden_3_112', 'Hidden_3_184', 'Hidden_3_176', 'Hidden_3_113',
       'Hidden_3_54', 'Hidden_3_213', 'Hidden_3_251', 'Hidden_3_100',
       'Hidden_3_109', 'Hidden_3_194', 'Hidden_3_13', 'Hidden_3_191',
       'Hidden_3_98', 'Hidden_3_45', 'Hidden_3_158', 'Hidden_3_209',
       'Hidden_3_169', 'Hidden_3_126', 'Hidden_3_221', 'Hidden_3_157',
       'Hidden_3_120', 'Hidden_3_102', 'Hidden_3_104', 'Hidden_3_65',
       'Hidden_3_52', 'Hidden_3_246', 'Hidden_3_214', 'Hidden_3_135',
       'Hidden_3_222', 'Hidden_3_245', 'Hidden_3_31', 'Hidden_3_211',
       'Hidden_3_198', 'Hidden_3_7', 'Hidden_3_180', 'Hidden_3_80',
       'Hidden_3_56', 'Hidden_3_9', 'Hidden_3_226', 'Hidden_3_207',
       'Hidden_3_200', 'Hidden_3_57', 'Hidden_3_196', 'Hidden_3_183',
       'Hidden_3_208', 